In [ ]:
!pip install selenium -q

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# driver = webdriver.Chrome()
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-css")
chrome_options.add_argument("--blink-settings=imageEnabled=false")
driver = webdriver.Chrome(options=chrome_options)
print("Running in headless mode.")

Running in headless mode.


In [3]:
links = pd.read_csv("./features.csv")

In [4]:
links

,url
0,https://ekantipur.com/feature/2024/09/12/a-per...
1,https://ekantipur.com/author/author-ReenaMokta...
2,https://ekantipur.com/feature/2024/09/10/enlig...
3,https://ekantipur.com/author/author-HRUMGzZ
4,https://ekantipur.com/feature/2024/09/09/woman...
...,...
5979,https://ekantipur.com/author/author-9196
5980,https://ekantipur.com/feature/2010/01/28/30585...
5981,https://ekantipur.com/author/author-9073
5982,https://ekantipur.com/feature/2010/01/19/30539...


In [4]:
# link = "https://ekantipur.com/news/2024/09/08/todays-weather-sometimes-light-sometimes-heavy-rain-42-44.html"

In [6]:
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

store = []

def scrape_p_tags(element: WebElement, data_str: str) -> str:
    # Find all <p> tags directly within the current element
    p_tags = element.find_elements(By.TAG_NAME, 'p')
    
    for p_tag in p_tags:
        if(p_tag.get_attribute("innerText") not in data_str):
            data_str += p_tag.get_attribute("innerText") + " "
    
    # Find all nested elements within the current element
    nested_elements = element.find_elements(By.XPATH, ".//*")
    
    for nested_element in nested_elements:
        if nested_element != element:  # Avoid re-processing the current element
            data_str = scrape_p_tags(nested_element, data_str)
    
    return data_str

# Create directory if it doesn't exist
path = "features"
if not os.path.exists(path):
    os.mkdir(path)

# ii = 1

for ii in range(0, len(links["url"])//3):
    # print(ii)
    link = links["url"][ii]
    if "author" not in link and link not in store:
        driver.get(link)
        store.append(link)

        content = driver.find_elements(By.CLASS_NAME, 'col-xs-7.mainContentX')
        if len(content) < 1:
            print(f"Some problem with link: {link}")
            continue
        
        # Process article header
        art = content[0].find_elements(By.CLASS_NAME, 'article-header')
        if len(art) < 1:
            print(f"Some problem with link: {link}")
            continue

        header = art[0].find_elements(By.TAG_NAME, 'h1')
        header_text = ""
        if len(header) > 0:
            header_text = header[0].get_attribute("innerText")
        
        # # Process sub-headlines
        # sub_head = content[0].find_elements(By.CLASS_NAME, "sub-headline")
        # sub_head_str = ""
        # for i in range(len(sub_head)):
        #     p_tag = sub_head[i].find_elements(By.TAG_NAME, 'p')
        #     for j in range(len(p_tag)):
        #         sub_head_str = sub_head_str + p_tag[j].get_attribute("innerText")+" "
        
        # txt2 = content[0].find_elements(By.CLASS_NAME, "description.current-news-block.portrait")
        # txt2_str = ""
        # for i in range(len(txt2)):
        #     p_tag = txt2[i].find_elements(By.TAG_NAME, 'p')
        #     for j in range(len(p_tag)):
        #         txt2_str = txt2_str + p_tag[j].get_attribute("innerText")+" "

        # Process description blocks
        txt = content[0].find_elements(By.CLASS_NAME, "col-md-12.portrait")
        txt_str = ""
        for i in range(len(txt)):
            p_tag = txt[i].find_elements(By.TAG_NAME, 'p')
            for j in range(len(p_tag)):
                txt_str = txt_str + p_tag[j].get_attribute("innerText")+" "
        
        # Save to file
        file_name = str(ii) + ".txt"
        file_path = os.path.join(path, file_name)
        with open(file_path, 'w') as f:
            f.write(header_text)
            f.write("\n\n")
            # f.write(sub_head_str)
            f.write("\n\n")
            # f.write(txt2_str)
            f.write(txt_str)
        
    ii += 1


In [5]:
driver.get(link)

##### Getting specific article tag

In [9]:
art = driver.find_elements(By.CLASS_NAME, 'article-header')
if len(art) < 1:
    print(f"Some problem with link: {link}")

In [10]:
art

[<selenium.webdriver.remote.webelement.WebElement (session="c44a346769814799ae13726e90746dac", element="f.1D3E6D6653E6020DFF11833A1A3FDE05.d.605479835DB6EF9B9DC5F8D71DE7A72B.e.854")>]

##### Extracting Headline in arcticle tag

In [6]:
# headline = art[0].find_elements(By.CSS_SELECTOR, '[data-component="headline-block"]')
# if len(headline) != 1:
#     print(f"Some problem with link: {link}")

In [18]:
header = art[0].find_elements(By.TAG_NAME, 'h1')
if len(header) != 1:
    print(f"Some problem with link: {link}")

header_text = header[0].get_attribute("innerText")

In [19]:
header_text

'आजको मौसम : कतै हल्का, कतै भारी वर्षा हुने'

##### Extracting context

In [24]:
data1 = driver.find_elements(By.CLASS_NAME, "description.current-news-block.portrait")
if len(data1) < 1:
    print(f"Some problem with link: {link}")

In [25]:
data1

[<selenium.webdriver.remote.webelement.WebElement (session="c44a346769814799ae13726e90746dac", element="f.1D3E6D6653E6020DFF11833A1A3FDE05.d.605479835DB6EF9B9DC5F8D71DE7A72B.e.18")>]

In [38]:
data_str = ""
for i in range(len(data1)):
    p_tag = data1[i].find_elements(By.TAG_NAME, 'p')
    for j in range(len(p_tag)):
        data_str = data_str + p_tag[j].get_attribute("innerText") + "\n"

In [39]:
data_str

'काठमाडौँ — हाल देशभर मनसुनी वायुको प्रभाव रहेको तथा यसको न्यूनचापीय रेखा सरदर स्थानको दक्षिणतिर अवस्थित रहेको जल तथा मौसम विज्ञान विभागले जनाएको छ\xa0।\n\nविभागका अनुसार, देशभर आंशिकदेखि साधारणतया बदली भई गण्डकी र कर्णाली प्रदेशका एक–दुई स्थानमा हल्का वर्षा भइरहेको छ\xa0। आज दिउँसो देशभर आंशिकदेखि साधारणतया बदली रहनेछ\xa0।\nकोशी, मधेश, वाग्मती, गण्डकी, लुम्बिनी, कर्णाली र सुदूरपश्चिम प्रदेशका केही स्थानमा मेघगर्जन र चट्याङसहित हल्कादेखि मध्यम वर्षाको सम्भावना छ\xa0। कोशी, वाग्मती र गण्डकी प्रदेशका एक–दुई स्थानमा भारी वर्षाको पनि सम्भावना छ\xa0।\nयसैगरी, आज राति देशभर आंशिकदेखि साधारणतया बदली रहने छभने कोशी प्रदेशको केही स्थान तथा बाँकी प्रदेशका पहाडी भागका केही स्थान र लुम्बिनी प्रदेशको तराई भू–भागका एक–दुई स्थानमा हल्कादेखि मध्यम वर्षाको सम्भावना छ\xa0।\nलुम्बिनी, मधेश, वाग्मती, गण्डकी, कर्णाली र सुदूरपश्चिम प्रदेशका केही स्थानमा मेघगर्जन एवं चट्याङसहित हल्कादेखि मध्यम वर्षाको सम्भावना छ\xa0। कोशी, वाग्मती र गण्डकी प्रदेशका एक–दुई स्थानमा भारी वर्षाको सम्भावना रहेको जनाउँदै विभागले त

In [28]:
data_str = data_str.strip()

##### Closing driver

In [7]:
driver.close()

##### Saving data

In [40]:
df = pd.DataFrame(columns=["url", "heading", "content"])

In [41]:
df.loc[len(df)] = [link, header_text, data_str]

In [42]:
df

,url,heading,content
0,https://ekantipur.com/news/2024/09/08/todays-w...,"आजको मौसम : कतै हल्का, कतै भारी वर्षा हुने",काठमाडौँ — हाल देशभर मनसुनी वायुको प्रभाव रहेक...


In [43]:
df.to_csv("dummy_data.csv", index=False)